In [63]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import matplotlib.pyplot as plt
from cassandra.cluster import Cluster
import requests

In [64]:
pip install cassandra-driver


Note: you may need to restart the kernel to use updated packages.


Connect to Cassandra

In [109]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

Create keyspace

In [66]:
keyspace_name = "data_quran"  


session.execute("CREATE KEYSPACE IF NOT EXISTS data_quran WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}")

session.set_keyspace('data_quran')

use keyspace

In [67]:
session.set_keyspace('data_quran')

create table

In [ ]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)

session.execute("""
    CREATE TABLE IF NOT EXISTS translation_data (
        translation_key text,
        sura_number int,
        data text,
        PRIMARY KEY (translation_key, sura_number)
    )
""")

Extract data from API

In [84]:
def insert_translation_data(translation_key, sura_number, data):
    session.execute("""
        INSERT INTO translation_data (translation_key, sura_number, data) 
        VALUES (%s, %s, %s)
    """, (translation_key, sura_number, data))

def fetch_and_store_translation_data(translation_key, sura_number):
    api_url = f"https://quranenc.com/api/v1/translation/sura/{translation_key}/{sura_number}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        insert_translation_data(translation_key, sura_number, str(data))


translation_key = "english_rwwad"

for sura_number in range(1, 115):
    fetch_and_store_translation_data(translation_key, sura_number)

Show Al-quran data 

In [112]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "translation_data"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")


for row in rows:
    print(row)  # 


Row(translation_key='english_rwwad', sura_number=1, data='{\'result\': [{\'id\': \'1\', \'sura\': \'1\', \'aya\': \'1\', \'arabic_text\': \'بِسۡمِ ٱللَّهِ ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ\', \'translation\': \'In the name of Allah, the Most Compassionate, the Most Merciful\', \'footnotes\': \'\'}, {\'id\': \'2\', \'sura\': \'1\', \'aya\': \'2\', \'arabic_text\': \'ٱلۡحَمۡدُ لِلَّهِ رَبِّ ٱلۡعَٰلَمِينَ\', \'translation\': \'All praise be to Allah[1], the Lord[2] of the worlds[3],\', \'footnotes\': \'[1] Allah is God’s most unique name.\\n [2] The Arabic term "rabb" (translated as "Lord") includes all of the following meanings: "owner, master, ruler, controller, sustainer, provider, guardian and caretaker."\\n [3] "the worlds" means mankind, jinn, and all that exists.\'}, {\'id\': \'3\', \'sura\': \'1\', \'aya\': \'3\', \'arabic_text\': \'ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ\', \'translation\': \'the Most Compassionate, the Most Merciful[4],\', \'footnotes\': \'[4] Ar-Rahmān and Ar-Raheem are two names of All

The Question

In [116]:
table_name = "translation_data"

# Define the search term
search_term = "battle"

# Execute a SELECT query to retrieve all rows from the table
query = f"SELECT sura_number, data FROM {table_name}"
rows = session.execute(query)

# Initialize a variable to accumulate the sum of sura numbers
total_sura_number = 0

# Iterate over the rows and check if the search term exists in the data
for row in rows:
    if search_term in row.data:
        print("Sura Number:", row.sura_number)
        total_sura_number += row.sura_number  # Add the sura_number to the total

# Print the total sum of sura numbers
print("Total Sum of Sura Numbers:", total_sura_number)


Sura Number: 2
Sura Number: 3
Sura Number: 4
Sura Number: 8
Sura Number: 9
Sura Number: 16
Sura Number: 18
Sura Number: 21
Sura Number: 33
Sura Number: 47
Sura Number: 52
Total Sum of Sura Numbers: 213
